# Deploy Traffic Flow Observed Devices in the IoT Agent, Context Broker, and Smart Village Platform

Now that all of our Smart Village microservices are running, we can start deploying smart devices to the platform. We will deploy an instance of the `TrafficFlowObserved` Kubernetes Custom Resource Definition (CRD) that we have defined in the Smart Village Operator. Run the command below to view the complete definition of the `sweden-veberod-1-lakaregatan-ne` `TrafficFlowObserved` smart device, then we will review each smaller section individually. 


In [1]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml


apiVersion: smartvillage.computate.org/v1
kind: TrafficFlowObserved
metadata:
  name: sweden-veberod-1-lakaregatan-ne
  namespace: smart-village-faeeb6c
spec:
  iotagent:
    name: iotagent-json
    namespace: smart-village-faeeb6c
    service_name: iotagent-json
  context_broker:
    name: scorpiobroker
    namespace: smart-village-faeeb6c
    service_name: scorpiobroker
  ngsi_ld:
    service: trafficflowobserveds
    service_path: /Sweden/Veberod/CityCenter
    context: https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld
  service_group:
    resource: /iot/json
    apikey: TrafficFlowObserved
  device:
    id: sweden-veberod-1-lakaregatan-ne
    subscription_url: http://ngsild-smartvillage-sync:8080
  message_broker:
    transport: AMQP
    host: default-rabbitmq
    port: 5672
    user: user
    secret:
      name: rabbitmq-password
      key: rabbitmq-password
  smartvillage:
    enabled: true
    auth_secret_name: keycloak-cli

### apiVersion, kind and metadata of the TrafficFlowObserved smart device

At the top, we define the `apiVersion`, `kind`, and `metadata` of the `TrafficFlowObserved` smart device. 

In [2]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^apiVersion: ' -A 3


apiVersion: smartvillage.computate.org/v1
kind: TrafficFlowObserved
metadata:
  name: sweden-veberod-1-lakaregatan-ne


We defined a Kubernetes resource with `apiVersion: smartvillage.computate.org/v1` and `kind: TrafficFlowObserved`. [See in the Smart Village Operator how we have defined a TrafficFlowObserved CRD](https://github.com/smartabyar-smartvillage/smartvillage-operator/blob/main/config/crd/bases/smartvillage.computate.org_trafficflowobserveds.yaml#L7-L15) with the same group, version, and kind. In the OpenShift environment it will have a name of `sweden-veberod-1-lakaregatan-ne` in our Developer Sandbox namespace. 

### iotagent configuration of the TrafficFlowObserved smart device

Then in the spec, we point the smart device to the IoT Agent base URL where the smart device will be registered. 

In [3]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  iotagent:' -B 1 -A 1


spec:
  iotagent:
    name: iotagent-json


### context_broker configuration of the TrafficFlowObserved smart device

We point the smart device to the Context Broker base URL where the smart device entity data can be found after the smart device is registered with the IoT Agent. 

In [4]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  context_broker:' -A 1


  context_broker:
    name: scorpiobroker


### ngsi_ld configuration of the TrafficFlowObserved smart device

We configure important NGSI-LD standard data for the smart device: 


In [5]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  ngsi_ld:' -A 3


  ngsi_ld:
    service: trafficflowobserveds
    service_path: /Sweden/Veberod/CityCenter
    context: https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld


- `service: trafficflowobserveds` is required so that entities for a given service can be held in a separate mongoDB database in the IoT Agent. 
- `service_path: /Sweden/Veberod/CityCenter` is required to differentiate between arrays of devices. 
- `context: https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld` is required to give specific documentation context to every NGSI-LD Entity and Attribute in the whole set of smart data. [See the NGSI-LD Context for the Smart Village Operator here](https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld) which contains links to every single Entity and Attribute in the Smart Village Operator. 

### device configuration of the TrafficFlowObserved smart device

Now we configure device specific configuration. 

In [6]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  device:' -A 2


  device:
    id: sweden-veberod-1-lakaregatan-ne
    subscription_url: http://ngsild-smartvillage-sync:8080


- We give the smart device the same `id` as the Kubernetes metadata name above to be consistent. 
- We set up a `subscription_url` where a webhook POST request will be made when the attributes of a TrafficFlowObserved Entity in the Context Broker change. We point this to the NGSI-LD Smart Village Sync application which is useful for sending smart device data from the Edge to the Cloud. 

### message_broker configuration of the TrafficFlowObserved smart device

Now we configure message broker specific configuration. 

In [7]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  message_broker:' -A 7


  message_broker:
    transport: AMQP
    host: default-rabbitmq
    port: 5672
    user: user
    secret:
      name: rabbitmq-password
      key: rabbitmq-password


Because we have a choice of two transport protocols (`AMQP`, and `MQTT`), we need to be very specific how we configure the message broker. 
- We use `transport: AMQP` in the Developer Sandbox because we don't have the operators in place to run Red Hat AMQ Broker which supports the MQTT 5 protocol. RabbitMQ supports an older version of MQTT, but the IoT Agents require MQTT 5. So we will simply use AMQP protocol which is supported by RabbitMQ and AMQ Broker. 
- We configure the `host: default-rabbitmq` and default AMQP `port: 5672` where RabbitMQ is accessible in your Developer Sandbox namespace. 
- We specify the message broker username `user: user`. 
- We specify the message broker password in the Kubernetes Secret with name `secret.name` and the password in the `secret.key` key. 

### smartvillage configuration of the TrafficFlowObserved smart device

Now we configure the Smart Village Platform specific configuration. 

In [8]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  smartvillage:' -A 3


  smartvillage:
    enabled: true
    auth_secret_name: keycloak-client-secret-smartvillage
    auth_token_url: https://sso.smartabyarsmartvillage.org/auth/realms/SMARTVILLAGE/protocol/openid-connect/token


- We configure the Kubernetes Secret with the name `auth_secret_name: smartvillage` where the Authentication `CLIENT_ID` and `CLIENT_SECRET` are stored to use the Smart Village API in your Developer Sandbox. 
- We configure the `auth_token_url` where the NGSI-LD Smart Village Sync application gets an auth token to send data to the Smart Village API in your Developer Sandbox. 
- We configure the `site_base_url` where the NGSI-LD Smart Village Sync application will send TrafficFlowObserved data into your Smart Village application in your Developer Sandbox. 

### attributes configuration of the TrafficFlowObserved smart device

Now we configure the Smart Village Platform specific configuration. 

In [9]:
%%bash
cat ~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml | grep '^  attributes:' -A 17


  attributes:
    entityId: sweden-veberod-1-lakaregatan-ne
    trafficSimulationId: TrafficSimulation-VeberodIntersection1
    laneAreaDetectorId: det_13
    location: 55.633703, 13.492540
    averageVehicleLength: 5
    averageGapDistance: 1
    averageVehicleSpeed: 55
    customRouteId: r42
    customSigma: 0.5
    customAcceleration: 2.6
    customDeceleration: 4.5
    customMinGreenTime: 33.00
    customMaxGreenTime: 39.00
    customAverageVehiclesPerMinute: 12.00
    customDemandScalingFactor: 1.00
    customQueueLengthThreshold: 8.00
    


- `trafficSimulationId` is a foreign key relation to our TrafficSimulation entity in the Smart Village Platform which describes our research project's whole traffic simulation in Veberöd Sweden. 
- `customTrafficLightId` is a foreign key relation to our SmartTrafficLight entity where we can collect data from all smart devices that make up a smart traffic light at one intersection. 
- `laneAreaDetectorId` is an identifier for a vehicle traffic camera in the SUMO Traffic Simulator, which is an open source software that generated our smart traffic light data. 
- `areaServed` is the GeoJSON coordinates of where the vehicle traffic camera is set up to observe the number of vehicles moving through the traffic light. 
- The rest of the attributes are various statistics to measure the flow of vehicles through the traffic light in a lane of traffic. 

### device configuration of the TrafficFlowObserved smart device

Now that all of our Smart Village microservices are running, we can deploy this TrafficFlowObserved smart device to the platform. 
- Our traffic flow observed smart device first gets configured into the IoT Agent JSON, 
- the IoT Agent sends a basic entity for the smart device to be saved in the Context Broker, 
- then we send an AMQP message to RabbitMQ with the current entity data we want to be loaded into the context broker, 
- the IoT Agent receives this entity data and converts it to NGSI-LD format and updates the entity in the context broker REST API. 
- The NGSI-LD Smart Village Sync application in the Smart Village Platform subscribes to all entity changes and receives a webhook API request from the context broker with the entity data. 
- The NGSI-LD Smart Village Sync application receives the context information, type, and ID of the entity, and queries the context broker for the latest entity data
- The NGSI-LD Smart Village Sync application then sends a PUT Import request with the TrafficFlowObserved model data to the Smarta Byar Smart Village REST API so that we can visualize the data in our cloud dashboard application. 

You can run the Ansible Playbook to perform all the actions above to deploy the TrafficFlowObserved smart device to the platform. 

In [ ]:
%%bash
ansible-playbook ~/smartvillage-operator/apply-trafficflowobserved.yaml \
  -e ansible_operator_meta_namespace=$(oc get project -o jsonpath={.items[0].metadata.name}) \
  -e crd_path=~/smartvillage-operator/kustomize/overlays/nerc-ocp-prod/ansible/trafficflowobserveds/sweden-veberod-1-lakaregatan-ne/trafficflowobserved.yaml
echo DONE

## Query the device in the IoT Agent

To query the IoT Agent, we need to include the device ID we specified in the `device` section of the CRD, but URL encoded in the IoT Agent URL. We also include the `FIWARE-Service` and `FIWARE-ServicePath` Headers from the `ngsi_ld` section of the CRD. 

In [ ]:
%%bash
DEVICE_ID='sweden-veberod-1-lakaregatan-ne' \
&& DEVICE_ID_URL_ENCODED=$(python -c "import urllib.parse;print(urllib.parse.quote('$DEVICE_ID'))") \
&& curl -s \
  -H 'FIWARE-Service: trafficflowobserveds' \
  -H 'FIWARE-ServicePath: /Sweden/Veberod/CityCenter' \
  "http://iotagent-json:4041/iot/devices/$DEVICE_ID_URL_ENCODED"

Notice that the IoT Agent keeps track of the `device_id`, `apikey`, `service`, `service_path`, `entity_name`, `entity_type`, `transport`, and all of the `attributes`. 

## Query the entity context data in the context broker

To query the Context Broker NGSI-LD API, we need to include the entity ID URL encoded. The entity ID always starts with `urn:ngsi-ld:`, followed by the type which is `TrafficFlowObserved`, followed by a `:`, followed by the device ID `sweden-veberod-1-lakaregatan-ne` from the `device` section of the CRD. We also include the `NGSILD-Tenant` which is the same as the `FIWARE-Service` header, and `NGSILD-Path` header which is the same as the `FIWARE-ServicePath` Header in the IoT Agent. 

In [ ]:
%%bash
ENTITY_ID='urn:ngsi-ld:TrafficFlowObserved:sweden-veberod-1-lakaregatan-ne' \
&& ENTITY_ID_URL_ENCODED=$(python -c "import urllib.parse;print(urllib.parse.quote('$ENTITY_ID'))") \
&& curl -s \
  -H 'NGSILD-Tenant: trafficflowobserveds' \
  -H 'NGSILD-Path: /Sweden/Veberod/CityCenter' \
  "http://scorpiobroker:9090/ngsi-ld/v1/entities/$ENTITY_ID_URL_ENCODED"

Notice that the id attribute is the same that was passed into the Context Broker URL. The "type" attribute provides as a value a link to documentation aboutTrafficFlowObserved data. The rest of the attributes are long URLs to MarkDown files in GitHub, with "type", "value" and other interesting context. These links to documentation for each attribute is the context of the entity. The basis of a context broker is to provide you with context for your data. So if you do not specifically provide the context in a header, the context broker will provide you with the context in a very verbose response. 

## Query the entity in the context broker with a link to the context

We'll use a similar request to the Context Broker as before, but include the NGSI-LD Context in the `Link` Header inside of `<` and `>` followed by the rest of the well-formed Link parameters for JSON-LD `; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"`. Using NGSI-LD requires a very strict and formal use of Headers that you may not be used to in web applications, at least I wasn't used to it yet when I was introduced. 

In [ ]:
%%bash
ENTITY_ID='urn:ngsi-ld:TrafficFlowObserved:sweden-veberod-1-lakaregatan-ne' \
&& ENTITY_ID_URL_ENCODED=$(python -c "import urllib.parse;print(urllib.parse.quote('$ENTITY_ID'))") \
&& curl -s \
  -H 'NGSILD-Tenant: trafficflowobserveds' \
  -H 'NGSILD-Path: /Sweden/Veberod/CityCenter' \
  -H 'Link: <https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"' \
  "http://scorpiobroker:9090/ngsi-ld/v1/entities/$ENTITY_ID_URL_ENCODED?options=keyValues"


by querying the context broker with the JSON-LD Context URL in the `Link` header, the context broker simplifies the response by returning attribute names instead of the full context URLs for each attribute. We also further simplify the response by adding the `?options=keyValues` query parameter to return the values for each attribute directly instead of a JsonObject as the value of each attribute. 

## Query the subscription in the context broker for the entity

To query the Context Broker NGSI-LD API, we need to include the subscription ID URL encoded. The subscription ID always starts with `urn:subscription:`, then we put the entity type which is `TrafficFlowObserved`, followed by a `:`, followed by the device ID `sweden-veberod-1-lakaregatan-ne` from the `device` section of the CRD. We also include the `NGSILD-Tenant` which is the same as the `FIWARE-Service` header, and `NGSILD-Path` header which is the same as the `FIWARE-ServicePath` Header in the IoT Agent. 

In [ ]:
%%bash
SUBSCRIPTION_ID='urn:subscription:TrafficFlowObserved-sweden-veberod-1-lakaregatan-ne' \
&& SUBSCRIPTION_ID_URL_ENCODED=$(python -c "import urllib.parse;print(urllib.parse.quote('$SUBSCRIPTION_ID'))") \
&& curl -s \
  -H 'NGSILD-Tenant: trafficflowobserveds' \
  -H 'NGSILD-Path: /Sweden/Veberod/CityCenter' \
  -H 'Link: <https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"' \
  "http://scorpiobroker:9090/ngsi-ld/v1/subscriptions/$SUBSCRIPTION_ID_URL_ENCODED"


by querying the context broker with the JSON-LD Context URL in the `Link` header, the context broker simplifies the response by returning attribute names instead of the full context URLs for each attribute. We also further simplify the response by adding the `?options=keyValues` query parameter to return the values for each attribute directly instead of a JsonObject as the value of each attribute. 